In [23]:

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START
from pydantic import BaseModel

In [24]:
from typing import Literal

In [25]:
class SupervisorDecision(BaseModel):
    next: Literal['researcher', 'coder', 'FINISH']

In [26]:
model = ChatOpenAI(model = 'gpt-5-2025-08-07')

In [27]:
model = model.with_structured_output(SupervisorDecision)

In [28]:
agents = ['researcher', 'coder']

In [29]:
system_prompt_part_1 = f""" 당신은 다음 서브에이전트 사이의 대화를 관리하는 슈퍼바이저입니다. 서브에이전트: {agents}. 
아래 사용자 요청에 따라, 다음으로 행동할 서브에이전트를 지목하세요. 각 서브에이전트는 임무를 수행하고 결과와 상태을 응답합니다. 
실행할 서브에이전트가 없거나 작업이 완료되면, FINISH로 응답하세요 """

In [30]:
system_prompt_part_2 = f""" 위 대화를 바탕으로, 다음으로 행동할 서브에이전트는 누구입니까? 아니면 FINISH 해야 합니까? 서브에이전트 :{', '.join(agents)}, FINISH"""

In [31]:
system_prompt_part_2

' 위 대화를 바탕으로, 다음으로 행동할 서브에이전트는 누구입니까? 아니면 FINISH 해야 합니까? 서브에이전트 :researcher, coder, FINISH'

In [32]:
system_prompt_part_1

" 당신은 다음 서브에이전트 사이의 대화를 관리하는 슈퍼바이저입니다. 서브에이전트: ['researcher', 'coder']. \n아래 사용자 요청에 따라, 다음으로 행동할 서브에이전트를 지목하세요. 각 서브에이전트는 임무를 수행하고 결과와 상태을 응답합니다. \n실행할 서브에이전트가 없거나 작업이 완료되면, FINISH로 응답하세요 "

In [33]:
def supervisor(state):
    messages = [
        ('system', system_prompt_part_1),
        *state['messages'], 
        ('system', system_prompt_part_2)
    ]
    return model.invoke(messages)

In [34]:
# 에이전트의 상태 정의
class AgentState(MessagesState):
    next: Literal['researcher', 'coder', 'FINISH']


In [35]:
def researcher(state : AgentState):
    response = {
        'role' : 'assistant', 
        'content' : "관련 데이터를 찾는 중입니다.... 잠시만 기다려주세요",
    }

    fake_data = {
        'data' : "전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]"
    }

    response['content'] += f"\n찾은 데이터: {fake_data['data']}"
    return {'messages' : [response]}

In [36]:
def coder(state:AgentState ):
    response = {
        'role' : 'assistant', 
        'content' : '코드를 작성 중입니다... 잠시만 기다려주세요'
    }
    fake_code = '''
def visualize_population(data):
    import matplotlib.pyplot as plt

    countries = list(data.keys())
    population = list(data.values())

    plt.bar(countries, population)
    plt.xlabel('Country')
    plt.ylabel('Population')
    plt.title('World Population by Country')
    plt.show()

data = {'USA': 331, 'China': 1400, 'India': 1300}
visualize_population(data)
'''
    response['content'] += f'\n작성된 코드:\n{fake_code}'
    return {'messages': [response]}

In [37]:
builder = StateGraph(AgentState)
builder.add_node('supervisor', supervisor)
builder.add_node('researcher', researcher),
builder.add_node('coder', coder),

builder.add_edge(START, 'supervisor')
builder.add_conditional_edges('supervisor'  , lambda state : state['next'])
builder.add_edge('researcher', "supervisor")
builder.add_edge('coder', 'supervisor')

graph = builder.compile()


In [38]:
graph.get_graph().draw_mermaid_png(output_file_path="./supervisor.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x89\x00\x00\x01\x08\x08\x02\x00\x00\x00%S\x9f\x97\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x1f\xdeIDATx\x9c\xed\xddw@\x13\xe7\xe3\x06\xf07\xe4\xb2\xd9 C" \n\xa2\x80\x80EE[\x0b\x16\xad\x03\x07\xaej\x1d(\xad\xb5Z\xabu\xafj\xdd\xd6\xe2\xa8\xab\xd6Z\xab\x96\xaau[\xfdj\x1dU\xd4:\xaaV- (*"\x82((\x84\x95@ !\xf9\xfd\x91\xfe(\xb5a\x88\t\xf7\x06\x9f\xcf_\xe1F\xee\xe1\x05\x9e\xdc]\xc2\x1dG\xab\xd5\x12\x00\x00\xca\x98\xb1\x1d\x00\x00@\x0ft\x13\x00\xd0\x08\xdd\x04\x004B7\x01\x00\x8d\xd0M\x00@#t\x13\x00\xd0\x88\xa9\xfd\xa2%\nu\xd6\xc3\xd2B\x99\xaa\xb4Dc\xccH&@(\xe6\xda8\xf2\\\x9a\x8b\xcc\xcc8lg\xa9YNfinV\x99\xa2P\xad.\xc3\xe7E\xaa\xc4\xf08\x12+\xc6\xce\x89o\xef"`;K\xcd4\x1amfJI\xde3\x95RQ\xcev\x96\x97\xc30\x1c\xb1\x15\xd7\xce\x89\xdfH*\xac~IN-?\xdft\xfbj\xe1\xdd\xebE\x1c.\xc7\xc9M\xac.{\xdd\xbb\x89cF\xb2\xd2JTJ\xcd;\xef7r\xa8i\x88\xd9u\xfe\xc0\xf3\xa2<\xb5\x99\x19\xc7\xd6Y\x88\x1f\\5\x18\x01G\xf6\xa4T\xa3\xd1Z\xd80!\x03\x1a\xb1\x1d\xa7:\xcf\x1

In [41]:
initial_state = {
    'messages': [
        {
            'role': 'user',
            'content': '전세계 인구를 국적을 기준으로 파이썬 코드로 시각화 해주세요.',
        }
    ],
    'next': 'supervisor',
}

In [43]:
for output in graph.stream(initial_state):
    node_name, node_result = list(output.items())[0]
    print(f'\n현재 노드: {node_name}')
    if node_result.get('messages'):
        print(f'응답: {node_result['messages'][-1]['content']}...')
    print(f'\n다음 단계: {node_result.get('next', 'N/A')}')


현재 노드: supervisor

다음 단계: researcher

현재 노드: researcher
응답: 관련 데이터를 찾는 중입니다.... 잠시만 기다려주세요
찾은 데이터: 전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]...

다음 단계: N/A

현재 노드: supervisor

다음 단계: coder

현재 노드: coder
응답: 코드를 작성 중입니다... 잠시만 기다려주세요
작성된 코드:

def visualize_population(data):
    import matplotlib.pyplot as plt

    countries = list(data.keys())
    population = list(data.values())

    plt.bar(countries, population)
    plt.xlabel('Country')
    plt.ylabel('Population')
    plt.title('World Population by Country')
    plt.show()

data = {'USA': 331, 'China': 1400, 'India': 1300}
visualize_population(data)
...

다음 단계: N/A

현재 노드: supervisor

다음 단계: researcher

현재 노드: researcher
응답: 관련 데이터를 찾는 중입니다.... 잠시만 기다려주세요
찾은 데이터: 전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]...

다음 단계: N/A

현재 노드: supervisor

다음 단계: researcher

현재 노드: researcher
응답: 관련 데이터를 찾는 중입니다.... 잠시만 기다려주세요
찾은 데이터: 전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]...

다음 단계: N/A

현재 노드: supervisor

다음 단계: researcher

현재 노드: researche

KeyboardInterrupt: 

In [44]:
from typing import Literal

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START
from pydantic import BaseModel


class SupervisorDecision(BaseModel):
    next: Literal['researcher', 'coder', 'FINISH']


# 모델 초기화
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
model = model.with_structured_output(SupervisorDecision)

# 사용 가능한 에이전트 정의
agents = ['researcher', 'coder']

# 시스템 프롬프트 정의
system_prompt_part_1 = f'''당신은 다음 서브에이전트 사이의 대화를 관리하는 슈퍼바이저입니다. 서브에이전트: {agents}. 아래 사용자 요청에 따라,  
다음으로 행동할 서브에이전트를 지목하세요. 각 서브에이전트는 임무를 수행하고 결과와 상태를 응답합니다. 실행할 서브에이전트가 없거나 작업이 완료되면,  
FINISH로 응답하세요.'''

system_prompt_part_2 = f'''위 대화를 바탕으로, 다음으로 행동할 서브에이전트는 누구입니까? 아니면 FINISH 해야 합니까? 서브에이전트: {', '.join(agents)}, FINISH'''


def supervisor(state):
    messages = [
        ('system', system_prompt_part_1),
        *state['messages'],
        ('system', system_prompt_part_2),
    ]
    return model.invoke(messages)


# 에이전트 상태 정의
class AgentState(MessagesState):
    next: Literal['researcher', 'coder', 'FINISH']


# 에이전트 함수 정의
def researcher(state: AgentState):
    # 실제 구현에서는 이 함수가 리서치 작업을 수행합니다.
    # 여기서는 임의로 관련 데이터를 찾는 척 합니다.
    response = {
        'role': 'assistant',
        'content': '관련 데이터를 찾는 중입니다... 잠시만 기다려주세요.',
    }
    # 임의의 데이터 생성
    fake_data = {
        'data': '전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]'
    }
    response['content'] += f'\n찾은 데이터: {fake_data['data']}'
    return {'messages': [response]}

def coder(state: AgentState):
    # 실제 구현에서는 이 함수가 코드를 작성합니다.
    # 여기서는 임의로 코드를 작성하는 척 합니다.
    response = {
        'role': 'assistant',
        'content': '코드를 작성 중입니다... 잠시만 기다려주세요.',
    }
    # 임의의 코드 생성
    fake_code = '''
def visualize_population(data):
    import matplotlib.pyplot as plt

    countries = list(data.keys())
    population = list(data.values())

    plt.bar(countries, population)
    plt.xlabel('Country')
    plt.ylabel('Population')
    plt.title('World Population by Country')
    plt.show()

data = {'USA': 331, 'China': 1400, 'India': 1300}
visualize_population(data)
'''
    response['content'] += f'\n작성된 코드:\n{fake_code}'
    return {'messages': [response]}


# 그래프 구축
builder = StateGraph(AgentState)
builder.add_node('supervisor', supervisor)
builder.add_node('researcher', researcher)
builder.add_node('coder', coder)

builder.add_edge(START, 'supervisor')
# 슈퍼바이저의 결정에 따라 에이전트 중 하나로 라우팅하거나 종료합니다.
builder.add_conditional_edges('supervisor', lambda state: state['next'])
builder.add_edge('researcher', 'supervisor')
builder.add_edge('coder', 'supervisor')

graph = builder.compile()

# 예시
initial_state = {
    'messages': [
        {
            'role': 'user',
            'content': '전세계 인구를 국적을 기준으로 시각화 해주세요.',
        }
    ],
    'next': 'supervisor',
}

for output in graph.stream(initial_state):
    node_name, node_result = list(output.items())[0]
    print(f'\n현재 노드: {node_name}')
    if node_result.get('messages'):
        print(f'응답: {node_result['messages'][-1]['content'][:100]}...')
    print(f'\n다음 단계: {node_result.get('next', 'N/A')}')


현재 노드: supervisor

다음 단계: researcher

현재 노드: researcher
응답: 관련 데이터를 찾는 중입니다... 잠시만 기다려주세요.
찾은 데이터: 전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]...

다음 단계: N/A

현재 노드: supervisor

다음 단계: coder

현재 노드: coder
응답: 코드를 작성 중입니다... 잠시만 기다려주세요.
작성된 코드:

def visualize_population(data):
    import matplotlib.pyplot as ...

다음 단계: N/A


Task supervisor with path ('__pregel_pull', 'supervisor') wrote to unknown channel branch:to:FINISH, ignoring it.



현재 노드: supervisor

다음 단계: FINISH
